In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('census_data.csv')

In [4]:
df.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
def compute_label(col):
    if col == ' <=50K':
        return 0
    else:
        return 1

In [6]:
df['income_bracket'] = df['income_bracket'].apply(compute_label)

In [7]:
df['income_bracket'].nunique()

2

In [8]:
X = df.drop('income_bracket',axis=1)

In [9]:
y = df['income_bracket']

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [12]:
df.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [13]:
#Categorical feature columns
workclass = tf.feature_column.categorical_column_with_hash_bucket('workclass',hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket('education',hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket('marital_status',hash_bucket_size=1000)
occupation = tf.feature_column.categorical_column_with_hash_bucket('occupation',hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket('relationship',hash_bucket_size=1000)
gender = tf.feature_column.categorical_column_with_vocabulary_list('gender',['Female','Male'])
native_country = tf.feature_column.categorical_column_with_hash_bucket('native_country',hash_bucket_size=1000)


In [14]:
#Continous feature columns
age = tf.feature_column.numeric_column('age')
education_num = tf.feature_column.numeric_column('education_num')
capital_gain = tf.feature_column.numeric_column('capital_gain')
capital_loss = tf.feature_column.numeric_column('capital_loss')
hours_per_week = tf.feature_column.numeric_column('hours_per_week')


In [18]:
feat_cols = [workclass,education,marital_status,occupation,relationship,gender,native_country,age,education_num,capital_gain,capital_loss,hours_per_week]

In [16]:
input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x = X_train,
                                                           y = y_train,
                                                           batch_size = 10,
                                                           num_epochs = 1000,
                                                           shuffle = True)

In [19]:
model = tf.estimator.LinearClassifier(feature_columns = feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\PRAMEE~1\\AppData\\Local\\Temp\\tmp6kqsw7xh', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
model.train(input_fn=input_func,steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\PRAMEE~1\AppData\Local\Temp\tmp6kqsw7xh\mo

In [21]:
pred_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test,
                                                        batch_size = len(X_test),
                                                       shuffle= False)

In [22]:
predictions = list(model.predict(input_fn = pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\PRAMEE~1\AppData\Local\Temp\tmp6kqsw7xh\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [24]:
final_pred = []

for pred in predictions:
    final_pred.append(pred['class_ids'][0])

In [26]:
final_pred[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [27]:
from sklearn.metrics import classification_report

In [29]:
print(classification_report(y_test,final_pred))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89      7462
           1       0.63      0.68      0.66      2307

    accuracy                           0.83      9769
   macro avg       0.77      0.78      0.77      9769
weighted avg       0.84      0.83      0.83      9769



In [30]:
eval_func =  tf.compat.v1.estimator.inputs.pandas_input_fn(x = X_test,
                                                           y = y_test,
                                                           batch_size = 10,
                                                           num_epochs = None,
                                                           shuffle = False)

In [33]:
eval_metrics = model.evaluate(input_fn=eval_func,steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-06-04T11:05:34Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\PRAMEE~1\AppData\Local\Temp\tmp6kqsw7xh\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1000]
INFO:tensorflow:Evaluation [400/1000]
INFO:tensorflow:Evaluation [500/1000]
INFO:tensorflow:Evaluation [600/1000]
INFO:tensorflow:Evaluation [700/1000]
INFO:tensorflow:Evaluation [800/1000]
INFO:tensorflow:Evaluation [900/1000]
INFO:tensorflow:Evaluation [1000/1000]
INFO:tensorflow:Inference Time : 9.05363s
INFO:tensorflow:Finished evaluation at 2021-06-04-11:05:43
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.8305, accuracy_baseline = 0.7628, auc = 0.88398045, auc_precision_recall = 0.6534708, average_l

In [34]:
eval_metrics

{'accuracy': 0.8305,
 'accuracy_baseline': 0.7628,
 'auc': 0.88398045,
 'auc_precision_recall': 0.6534708,
 'average_loss': 0.59921473,
 'label/mean': 0.2372,
 'loss': 0.5992148,
 'precision': 0.63269305,
 'prediction/mean': 0.28574297,
 'recall': 0.68043846,
 'global_step': 5000}

In [35]:
dnn_model = tf.estimator.DNNClassifier(hidden_units = [12,12,12],feature_columns = feat_cols,n_classes = 2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\PRAMEE~1\\AppData\\Local\\Temp\\tmpm7btyknz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [36]:
dnn_model.train(input_fn = input_func,steps=1000)

INFO:tensorflow:Calling model_fn.


ValueError: Items of feature_columns must be a <class 'tensorflow.python.feature_column.feature_column_v2.DenseColumn'>. You can wrap a categorical column with an embedding_column or indicator_column. Given: HashedCategoricalColumn(key='education', hash_bucket_size=1000, dtype=tf.string)

In [37]:
embeded_workclass = tf.feature_column.embedding_column(workclass,dimension=1000)
embeded_education = tf.feature_column.embedding_column(education,dimension=1000)
embeded_marital_status = tf.feature_column.embedding_column(marital_status,dimension=1000)
embeded_occupation = tf.feature_column.embedding_column(occupation,dimension=1000)
embeded_relationship = tf.feature_column.embedding_column(relationship,dimension=1000)
embeded_gender = tf.feature_column.embedding_column(gender,dimension=2)
embeded_native_country = tf.feature_column.embedding_column(native_country,dimension=1000)


In [38]:
feat_cols_embeded = [embeded_workclass,embeded_education,embeded_marital_status,embeded_occupation,embeded_relationship,embeded_gender,embeded_native_country,age,education_num,capital_gain,capital_loss,hours_per_week]

In [39]:
input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x = X_train,
                                                 y = y_train,
                                                 batch_size = 10,
                                                 num_epochs = 1000,
                                                 shuffle = True)

In [40]:
dnn_model = tf.estimator.DNNClassifier(hidden_units = [12,12,12],feature_columns = feat_cols_embeded,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\PRAMEE~1\\AppData\\Local\\Temp\\tmpwpw0e8wq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [41]:
dnn_model.train(input_fn = input_func,steps=1000)

INFO:tensorflow:Calling model_fn.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\PRAMEE~1\AppData\Local\Temp\tmpwp

In [42]:
pred_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test,
                                                        batch_size = len(X_test),
                                                       shuffle= False)

In [43]:
dnn_predictions =  list(dnn_model.predict(input_fn = pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\PRAMEE~1\AppData\Local\Temp\tmpwpw0e8wq\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [45]:
final_pred_dnn = []

for pred in dnn_predictions:
    final_pred_dnn.append(pred['class_ids'][0])

In [46]:
print(classification_report(y_test,final_pred_dnn))

              precision    recall  f1-score   support

           0       0.80      0.96      0.87      7462
           1       0.62      0.22      0.32      2307

    accuracy                           0.78      9769
   macro avg       0.71      0.59      0.60      9769
weighted avg       0.76      0.78      0.74      9769

